In [1]:
from src.augmentation.augment import Augmentation
from src.selection.select import Selection
from src.utils.alter import Alter

from ptrail.core.Datasets import Datasets
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.preprocessing.statistics import Statistics
from ptrail.preprocessing.filters import Filters

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from random import *

In [2]:
def trajectoryAugumentationProcedure(trajs, seed, n, k, pradius, model):
    myRandom = Random(seed * (n * k * pradius))
    #     Split data into training and testing
    splits = Selection.select_randomly(trajs, myRandom, seed)
    paramTestingDataSet = Filters.remove_duplicates(dataframe=trajs)

    trainDataParm = paramTestingDataSet.loc[paramTestingDataSet.traj_id.isin(splits["train"]) == True].dropna()
    testDataParm = paramTestingDataSet.loc[paramTestingDataSet.traj_id.isin(splits["test"]) == True].dropna()
    #  Organize test data
    testData = PTRAILDataFrame(data_set=testDataParm,
                               latitude='lat',
                               longitude='lon',
                               datetime='DateTime',
                               traj_id='traj_id')
    #Calculate features for testing data
    statsTestParm = Statistics.generate_kinematic_stats(dataframe=testData,
                                                target_col_name='Species')
    pivotedStatsTestParm = Statistics.pivot_stats_df(dataframe=statsTestParm,
                                              target_col_name='Species')
    #Remove duplicate columns
    pivotedStatsTestParm = pivotedStatsTestParm.loc[:,~pivotedStatsTestParm.columns.duplicated()]

    testParmX = pivotedStatsTestParm.drop(columns='Species')
    testParmY = pivotedStatsTestParm['Species'].to_numpy()

    # Organize training data
    noiseTraj = trainDataParm['traj_id'].unique()
    #Select n % of training data to augment
    ### Here you are selecting without replacement, but the same trajectory could be
    ### selected more than once according to the data augumentation theory I believe

    #Using rnaomd.choices instead of sample has replacement
    sampledTraj = myRandom.choices(sorted(noiseTraj), k=math.floor(n * len(noiseTraj)))
    for traj in sampledTraj:
        trajToChange = trainDataParm.loc[trainDataParm.traj_id == traj]
        #Add noise to selected data then concatenate it to the training data
        trajChanged = Augmentation.augment_trajectories_with_randomly_generated_points(trajToChange, pradius,
                                                                                       k, 100, myRandom,)
        trainDataParm = pd.concat([trainDataParm, trajChanged], ignore_index = True)

    trainDataNoiseFilt = trainDataParm.filter(["traj_id", "DateTime","lat", "lon", "StarkeyTime", "GMDate", "GMTime", "LocDate", "LocTime", "RadNum", "Species", "UTME", "UTMN", "Year", "Grensunr", "Grensuns", "Obswt"])



    trainDataNoise = PTRAILDataFrame(data_set=trainDataNoiseFilt,
                                            datetime='DateTime',
                                            traj_id='traj_id',
                                            latitude='lat',
                                            longitude='lon')

    #Calculate features of training data
    statsTrainNoiseParm = Statistics.generate_kinematic_stats(dataframe=trainDataNoise,
                                                        target_col_name='Species')

    pivotedStatsTrainNoiseParm = Statistics.pivot_stats_df(dataframe=statsTrainNoiseParm,
                                                      target_col_name='Species')

    pivotedStatsTrainNoise = pivotedStatsTrainNoiseParm.loc[:,~pivotedStatsTrainNoiseParm.columns.duplicated()]

    pivotedStatsTrainNoise=pivotedStatsTrainNoise.dropna()

    trainParmX = pivotedStatsTrainNoise.drop(columns='Species')
    trainParmY = pivotedStatsTrainNoise['Species'].to_numpy()

    #Test model classification
#     rf_model = RandomForestClassifier(random_state=seed)
    model.fit(trainParmX, trainParmY)
    test_predict = model.predict(testParmX)
    performance_val = f1_score(testParmY, test_predict, average='weighted')
    print(f"Current run: k={k}, pradius={pradius}, n={n}, fscore={performance_val}, seed={seed}")
    return str(f"{n},{k},{pradius},{performance_val}\n")

In [3]:
# Import the dataset.
dataset = Datasets.load_starkey()
ready_dataset = Filters.remove_duplicates(dataframe=dataset)

seeds = [14159,26535,89793,23846,26433,83279,50288,41971,69399,37510]
n_vals = [.2, .3, .4, .5]
k_vals = [.1, .2, .3, .4]
rad_vals = [.001, .005, .01, .02]

# Repeats 10 times (10 seeds) 80% training/ 20% testing

text_file = open("results.txt", "w")
for n in n_vals:
    for k in k_vals:
        for rad in rad_vals:
            for s in seeds:
                # n = percentage of trajs to be augumented, k = percentage of points to be changed, and radius
                print(trajectoryAugumentationProcedure(ready_dataset, s, n, k,
                                                       rad, RandomForestClassifier(random_state=s)))
text_file.close()

------------------------ Dataset Facts ------------------------------

Number of unique Trajectories in the data: 253
Number of points in the data: 287136
Dataset time range: 1196 days 22:51:45
Datatype of the DataFrame: <class 'ptrail.core.TrajectoryDF.PTRAILDataFrame'>
Dataset Bounding Box: (45.18896978643169, -118.61020848239596, 45.314545642992, -118.50455596234036)

---------------------------------------------------------------------
253 183
252 167
251 166
250 88
249 242
248 55
247 118
246 240
245 203
244 163
243 160
242 214
241 223
240 55
239 15
238 69
237 84
236 221
235 108
234 172
233 56
232 4
231 46
230 28
229 214
228 26
227 69
226 122
225 98
224 151
223 61
222 201
221 117
220 84
219 48
218 47
217 112
216 15
215 57
214 22
213 144
212 3
211 61
210 98
209 30
208 135
207 44
206 34
205 129
204 130
203 178
202 2
201 89
200 173
199 114
198 29
197 181
196 60
195 17
194 144
193 160
192 182
191 186
190 0
189 161
188 161
187 161
186 177
185 45
184 101
183 38
182 128
181 24
180 1
179 3

ValueError: empty range for randrange()